In [5]:
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""

In [1]:
#Using boto
import boto3

In [6]:
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource('dynamodb', region_name='us-west-2', aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'), aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'))
orders = dynamodb.Table('orders')

In [7]:
orders_det ={'order_id': 6,
        'customer_id':  104,
        'payment_method': 'Cash',
        'order_date':'2020-09-28'}

In [8]:
#Inserting data into dynamoDB
response = orders.put_item(Item=orders_det)

In [9]:
#reading data from dynamoDB
respone = orders.get_item(Key={"order_id":6})
print(respone)

{&#39;Item&#39;: {&#39;customer_id&#39;: Decimal(&#39;104&#39;), &#39;order_date&#39;: &#39;2020-09-28&#39;, &#39;order_id&#39;: Decimal(&#39;6&#39;), &#39;payment_method&#39;: &#39;Cash&#39;}, &#39;ResponseMetadata&#39;: {&#39;RequestId&#39;: &#39;993S7N2TU90Q4IRAS0KGOM1S7NVV4KQNSO5AEMVJF66Q9ASUAAJG&#39;, &#39;HTTPStatusCode&#39;: 200, &#39;HTTPHeaders&#39;: {&#39;server&#39;: &#39;Server&#39;, &#39;date&#39;: &#39;Fri, 02 Oct 2020 01:24:32 GMT&#39;, &#39;content-type&#39;: &#39;application/x-amz-json-1.0&#39;, &#39;content-length&#39;: &#39;119&#39;, &#39;connection&#39;: &#39;keep-alive&#39;, &#39;x-amzn-requestid&#39;: &#39;993S7N2TU90Q4IRAS0KGOM1S7NVV4KQNSO5AEMVJF66Q9ASUAAJG&#39;, &#39;x-amz-crc32&#39;: &#39;1686246438&#39;}, &#39;RetryAttempts&#39;: 0}}


# https://docs.aws.amazon.com/amazondynamodb/latest/APIReference/API_PutItem.html

In [2]:
#AccessKey Check
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
if access_key is None or secret_key is None:
    print('No access key is available.')
    sys.exit()

In [13]:
#Using python request
import sys, os, base64, datetime, hashlib, hmac 
import requests 

method = 'POST'
service = 'dynamodb'
host = 'dynamodb.us-west-2.amazonaws.com'
region = 'us-west-2'
endpoint = 'https://dynamodb.us-west-2.amazonaws.com/'
payload =  '{"TableName": "orders","Key": {"order_id": {"N": "4"}}}'
content_type = 'application/json'
amz_target = 'DynamoDB_20120810.GetItem'
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

def sign(key, msg):
    return hmac.new(key, msg.encode("utf-8"), hashlib.sha256).digest()

def getSignatureKey(key, date_stamp, regionName, serviceName):
    kDate = sign(('AWS4' + key).encode('utf-8'), date_stamp)
    kRegion = sign(kDate, regionName)
    kService = sign(kRegion, serviceName)
    kSigning = sign(kService, 'aws4_request')
    return kSigning

t = datetime.datetime.utcnow()
req_dt = t.strftime('%Y%m%dT%H%M%SZ')
date_stamp = t.strftime('%Y%m%d')


canonical_uri = '/'
canonical_querystring = ''
canonical_headers = 'content-type:' + content_type + '\n' + 'host:' + host + '\n' + 'x-amz-date:' + req_dt + '\n' + 'x-amz-target:' + amz_target + '\n'

signed_headers = 'content-type;host;x-amz-date;x-amz-target'
payload_hash = hashlib.sha256(payload.encode('utf-8')).hexdigest()
canonical_request = method + '\n' + canonical_uri + '\n' + canonical_querystring + '\n' + canonical_headers + '\n' + signed_headers + '\n' + payload_hash

algorithm = 'AWS4-HMAC-SHA256'
credential_scope = date_stamp + '/' + region + '/' + service + '/' + 'aws4_request'
string_to_sign = algorithm + '\n' +  req_dt + '\n' +  credential_scope + '\n' +  hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()
signing_key = getSignatureKey(secret_key, date_stamp, region, service)
signature = hmac.new(signing_key, (string_to_sign).encode('utf-8'), hashlib.sha256).hexdigest()
authorization_header = algorithm + ' ' + 'Credential=' + access_key + '/' + credential_scope + ', ' +  'SignedHeaders=' + signed_headers + ', ' + 'Signature=' + signature

headers = {'Content-Type':content_type,
           'X-Amz-Date':req_dt,
           'X-Amz-Target':amz_target,
           'Authorization':authorization_header}

r = requests.post(endpoint, data=payload, headers=headers)
print('Response code: %d\n' % r.status_code)
print(r.text)

Response code: 200

{&quot;Item&quot;:{&quot;customer_id&quot;:{&quot;N&quot;:&quot;101&quot;},&quot;order_date&quot;:{&quot;S&quot;:&quot;2020-09-20&quot;},&quot;order_id&quot;:{&quot;N&quot;:&quot;4&quot;},&quot;payment_method&quot;:{&quot;S&quot;:&quot;Cash&quot;}}}
